In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import sqlite3
import pandas as pd
import random
import re
random.seed(1)
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from textblob import Word

import string
from time import time

from bs4 import BeautifulSoup  
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects\venv\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cur_dir = os.getcwd()
print("Current directory: {}".format(cur_dir))
# list files in the current directory
#print("Files in the current directory: {}".format(os.listdir(cur_dir)))
data_path = cur_dir+"/drugsCom"
train_path = data_path + "/drugsComTrain_raw.tsv"
test_path = data_path + "/drugsComTest_raw.tsv"
#rawdata_path = cur_dir+"/drugsCom"
#metadata_path = cur_dir+"/meta_Health_and_Personal_Care.json.gz"
print("Path of the raw data: {}".format(data_path))
print("Files in the drugCom directory: {}".format(os.listdir(data_path)))

Current directory: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects
Path of the raw data: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects/drugsCom
Files in the drugCom directory: ['drug-enforcement-0001-of-0001.json', 'drug-enforcement-0001-of-0001.json.zip', 'drugsComTest_raw.tsv', 'drugsComTrain_raw.tsv', 'drugs_recalls_fda.tsv', 'FDA_Drug_Recalls_2012_to_2016_by_Recall_Class.csv', 'fda_med_devices_recalls_2014_to_2017.tsv']


In [4]:
train = pd.read_table(train_path, sep='\t')
test = pd.read_table(test_path, sep='\t')

In [5]:
train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [13]:
train.shape

(161297, 7)

In [14]:
train['drugName'].nunique()

3436

In [8]:
documents = train['review'].values

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
documents_train, documents_test = train_test_split(documents, test_size = 0.2, random_state = 42)

In [11]:
# clean the data set

verb_exp = ['VB', 'VBZ', 'VBP', 'VBD','VBN','VBG']
#porter_stemmer = PorterStemmer()
def clean_text(row):
    soup = BeautifulSoup(row, 'html.parser')
    #remove code
    for tag in soup.find_all('code'):
        tag.replaceWith(' ')
        
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    no_link_email_space = "".join([ch for ch in no_link_email if ch not in set(tab_text)])
    #remove fomula
    reg = '(\$.+?\$)|((\\\\begin\{.+?\})(.+?)(\\\\end\{(.+?)\}))'
    raw = re.sub(reg, "", no_link_email_space, flags=re.IGNORECASE)   
    return(raw)

def lem_stop(row):
    row = row.lower()
    #remove numbers
    raw = re.sub('[0-9]+?', ' ', row) 
    # remove punctuation
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    raw = regex.sub(' ', raw)
    #clean out the characters left out after the above step, like we’re, I’m, It’s, i.e.
    raw = re.sub('( s )|( re )|( m )|( i e )',' ',raw) 
    # lementize
    row_t = TextBlob(raw)
    raw = []
    for word, pos in row_t.tags:
        if pos in verb_exp:
            word = Word(word)
            word = word.lemmatize("v")
        else:
            word = Word(word)
            word = word.lemmatize()
        raw.append(word)
    clean = ' '.join(raw)      
    stop_words = set(stopwords.words('english'))
    # remove stop words
    cleaned_text = " ".join([word for word in word_tokenize(clean) if word not in stop_words])      
    return(cleaned_text)

In [14]:
# get the clean review
review_rm = train['review'].map(lambda i: clean_text(i))

In [15]:
review_clean = review_rm.map(lambda i: lem_stop(i))

In [16]:
def sentiment_ana(i):
    i_tr = TextBlob(i)
    return(i_tr.sentiment[0], i_tr.sentiment[1])

In [17]:

sentiment = review_clean.map(lambda i: sentiment_ana(i))


In [53]:
print(sentiment)

0                                          (0.0, 0.0)
1                    (0.18802083333333336, 0.4828125)
2          (0.13848484848484852, 0.32747474747474753)
3           (0.35714285714285715, 0.6071428571428571)
4           (0.17233333333333328, 0.3896666666666667)
5          (0.10208333333333333, 0.28541666666666665)
6           (0.1111111111111111, 0.16666666666666666)
7                      (0.059166666666666666, 0.2825)
8                           (0.2, 0.5166666666666666)
9                      (0.04375000000000002, 0.34375)
10           (0.21000000000000002, 0.653888888888889)
11                       (0.2525, 0.4583333333333333)
12          (-0.17777777777777773, 0.611111111111111)
13        (-0.036574074074074064, 0.5416666666666665)
14         (0.05286796536796535, 0.49983766233766236)
15          (0.19722222222222227, 0.5981481481481482)
16        (-0.038636363636363635, 0.4212121212121212)
17         (0.30833333333333335, 0.47222222222222215)
18           (0.179166666666

In [19]:
senti_pol, senti_sub = [list(t) for t in zip(*sentiment)]

In [20]:
print(review_clean.shape)

(161297,)


In [21]:
train['Sentimental_Polarity'] = senti_pol
train['Sentimental_Subjectivity'] = senti_sub

In [22]:
train.shape

(161297, 9)

In [23]:
train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,Sentimental_Polarity,Sentimental_Subjectivity
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,0.000000,0.000000
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,0.188021,0.482812
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,0.138485,0.327475
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,0.357143,0.607143
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,0.172333,0.389667


Now add the recall by combining data from fda dataset

In [24]:
cur_dir = os.getcwd()
data_path = cur_dir+"/drugsCom"
fda_2016_path = data_path + "/drugs_recalls_fda.tsv"
fda_2014_path = data_path + "/FDA_Drug_Recalls_2012_to_2016_by_Recall_Class.csv"
fda_2016_onwards = pd.read_csv(fda_2016_path, parse_dates = [0], sep='\t')
fda_2012_to_2016 = pd.read_csv(fda_2014_path, parse_dates=[0], sep=',')

fda_2012_to_2016['Date'] = fda_2012_to_2016['Recall Initiation Date']
fda_2012_to_2016['Brand'] = fda_2012_to_2016['Recalling Firm']
fda_2012_to_2016['Reason / Problem'] = fda_2012_to_2016['Reason for Recall']
fda_2012_to_2016['Company'] = fda_2012_to_2016['Recalling Firm']

all_fda_data = pd.concat([fda_2016_onwards, \
                              fda_2012_to_2016[['Date', 'Brand', \
                                                'Product Description', 'Reason / Problem', 'Company']]])


In [25]:
train_unique_drugs = train['drugName'].unique()
len(train_unique_drugs)
train_unique_drugs[1]

'Guanfacine'

In [26]:
total_products_recalled = 0
for online_drug in train_unique_drugs:
    total_products_recalled += all_fda_data['Product Description'].str.contains(online_drug, regex=False).sum()

print(total_products_recalled)

3529


In [27]:
print (train_unique_drugs.size)

3436


In [28]:
import datetime
import dateutil.parser
recallstatus = []
for drugName, dateValue in zip(train['drugName'],train['date']):
    recalled_drugs = all_fda_data['Product Description'].str.contains(drugName.split(' / ')[0], regex=False)
    if any(recalled_drugs):
        recallDateTime = str(all_fda_data[recalled_drugs]['Date'].iloc[0])
        recallDateTime = recallDateTime.replace("/", "-")
        if dateutil.parser.parse(dateValue) < \
                dateutil.parser.parse(recallDateTime):
            recallstatus.append(True)
        else:
            recallstatus.append(False)
    else:
        recallstatus.append(False)

In [29]:

train['recall_status'] = recallstatus
# train.drop('recall_status', axis=1, inplace=True)

In [30]:
print(train.shape)

(161297, 10)


In [31]:
train_recall_df = train.loc[train['recall_status'] == True]
print(train_recall_df.shape)

(31256, 10)


In [32]:
train_nonrecall_df = train.loc[train['recall_status'] == False]
print(train_nonrecall_df.shape)

(130041, 10)


In [157]:
train_recall_df.to_csv("drugsTrainingDataRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [158]:
train_nonrecall_df.to_csv("drugsTrainingDataNonRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [160]:
train.loc[(train['Sentimental_Polarity'] < 0.02)].shape

(66880, 10)

In [161]:
train_recall_df.shape

(31256, 10)

In [163]:
train_recall_df.loc[(train_recall_df['Sentimental_Polarity'] < 0.05)].shape

(13387, 10)

In [164]:
train_recall_df.loc[(train_recall_df['Sentimental_Polarity'] > 0.05) & (train_recall_df['usefulCount'] > 20)].shape

(11230, 10)

In [165]:
train_nonrecall_df.shape

(130041, 10)

In [166]:
train_nonrecall_df.loc[(train_nonrecall_df['Sentimental_Polarity'] > 0.09)].shape

(55513, 10)

In [167]:
train_nonrecall_df.loc[(train_nonrecall_df['Sentimental_Polarity'] <= 0.09)].shape

(74528, 10)

In [168]:
train.to_csv("drugsTrainingDataWithFeatures", sep='\t', encoding='utf-8')

Run sentimental analysis on Test Data i.e. drugsComTest

In [41]:
# get the clean review
review_rm_test = test['review'].map(lambda i: clean_text(i))

In [42]:
review_clean_test = review_rm_test.map(lambda i: lem_stop(i))

In [49]:
def sentiment_ana(i):
    i_tr = TextBlob(i)
    return(i_tr.sentiment[0], i_tr.sentiment[1])

In [50]:
sentiment_test = review_clean_test.map(lambda i: sentiment_ana(i))

In [51]:
senti_pol_test, senti_sub_test = [list(t) for t in zip(*sentiment_test)]

In [52]:
test['Sentimental_Polarity'] = senti_pol_test
test['Sentimental_Subjectivity'] = senti_sub_test

In [53]:
test.shape

(53766, 9)

In [171]:
import dateutil.parser
test_recallstatus = []
for drugName, dateValue in zip(test['drugName'],test['date']):
    recalled_drugs = all_fda_data['Product Description'].str.contains(drugName.split(' / ')[0], regex=False)
    if any(recalled_drugs):
        recallDateTime = str(all_fda_data[recalled_drugs]['Date'].iloc[0])
        recallDateTime = recallDateTime.replace("/", "-")
        if dateutil.parser.parse(dateValue) < \
                dateutil.parser.parse(recallDateTime):
            test_recallstatus.append(True)
        else:
            test_recallstatus.append(False)
    else:
        test_recallstatus.append(False)

print(len(test_recallstatus))

53766


In [172]:
# series_data = pd.Series(test_recallstatus)
test['recall_status'] = test_recallstatus

In [173]:
# test['recall_status'] = test_recallstatus
# test.drop('recall_status', axis=1, inplace=True)
print(test.shape)
# test.head()

(53766, 10)


In [174]:
test_recall_df = train.loc[train['recall_status'] == True]
print(test_recall_df.shape)

(31256, 10)


In [175]:
test_nonrecall_df = train.loc[train['recall_status'] == False]
print(test_nonrecall_df.shape)

(130041, 10)


In [176]:
test_recall_df.to_csv("drugsTestDataRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [177]:
test_nonrecall_df.to_csv("drugsTestDataNonRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [179]:
test.to_csv("drugsTestingDataWithFeatures", sep='\t', encoding='utf-8')

In [33]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('vader_lexicon')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:

def VaderAnalysis(inputstring):
    all_sent = sent_tokenize(inputstring)
    totalPositiveCount = 0
    totalNegativeCount = 0
    totalNeutralCount = 0
    totalCompountCount = 0
    totalSentenceCount = 0 
    averagePositiveScore = 0
    averageNegativeScore = 0
    averageNeutralScroe = 0
    averageCompountCount = 0
    for i in range(len(all_sent)):
        sentence = all_sent[i]
        snt = analyser.polarity_scores(sentence)
        totalPositiveCount = totalPositiveCount + snt['pos']
        totalNegativeCount = totalNegativeCount + snt['neg']
        totalNeutralCount = totalNeutralCount + snt['neu']
        totalCompountCount = totalCompountCount + snt['compound']
        totalSentenceCount = totalSentenceCount + 1
    if totalSentenceCount > 0:
        averagePositiveScore = totalPositiveCount/totalSentenceCount
        averageNegativeScore = totalNegativeCount/totalSentenceCount
        averageNeutralScroe = totalNeutralCount/totalSentenceCount
        averageCompountCount = totalCompountCount/totalSentenceCount
    return (averageNegativeScore, averageNeutralScroe, averagePositiveScore, averageCompountCount)

In [35]:
# clean the data set

verb_exp = ['VB', 'VBZ', 'VBP', 'VBD','VBN','VBG']
#porter_stemmer = PorterStemmer()
def clean_text_va(row):
    soup = BeautifulSoup(row, 'html.parser')
    #remove code
    for tag in soup.find_all('code'):
        tag.replaceWith(' ')
        
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    no_link_email_space = "".join([ch for ch in no_link_email if ch not in set(tab_text)])
    #remove fomula
    reg = '(\$.+?\$)|((\\\\begin\{.+?\})(.+?)(\\\\end\{(.+?)\}))'
    raw = re.sub(reg, "", no_link_email_space, flags=re.IGNORECASE)   
    return(raw)

def lem_stop_va(row):
    row = row.lower()
    #remove numbers
    raw = re.sub('[0-9]+?', ' ', row) 
    # # remove punctuation
    # regex = re.compile('[%s]' % re.escape(string.punctuation))
    # raw = regex.sub(' ', raw)
    #clean out the characters left out after the above step, like we’re, I’m, It’s, i.e.
    raw = re.sub('( s )|( re )|( m )|( i e )',' ',raw) 
    # lementize
    row_t = TextBlob(raw)
    raw = []
    for word, pos in row_t.tags:
        if pos in verb_exp:
            word = Word(word)
            word = word.lemmatize("v")
        else:
            word = Word(word)
            word = word.lemmatize()
        raw.append(word)
    clean = ' '.join(raw)      
    stop_words = set(stopwords.words('english'))
    # remove stop words
    cleaned_text = " ".join([word for word in word_tokenize(clean) if word not in stop_words])      
    return(cleaned_text)

In [36]:
review_va_clean = train['review'].map(lambda i: clean_text_va(i))
review_va_stop = review_va_clean.map(lambda i: lem_stop_va(i))

In [37]:
vaderAnalysisData = review_va_stop.map(lambda i: VaderAnalysis(i))

In [39]:
senti_neg, senti_neu, senti_pos, senti_compount = [list(t) for t in zip(*vaderAnalysisData)]

In [40]:
train['vader_neg'] = senti_neg
train['vader_neu'] = senti_neu
train['vader_pos'] = senti_pos
train['vader_comp'] = senti_compount

In [41]:
train.shape

(161297, 14)

In [227]:
train_recall_df = train.loc[train['recall_status'] == True]
print(train_recall_df.shape)

(31256, 14)


In [228]:
train_nonrecall_df = train.loc[train['recall_status'] == False]
print(train_nonrecall_df.shape)

(130041, 14)


In [229]:
train_recall_df.to_csv("drugsTrainingDataRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [230]:
train_nonrecall_df.to_csv("drugsTrainingDataNonRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [231]:
train.to_csv("drugsTrainingDataWithFeatures", sep='\t', encoding='utf-8')

In [42]:
review_va_clean_test = test['review'].map(lambda i: clean_text_va(i))
review_va_stop_test = review_va_clean_test.map(lambda i: lem_stop_va(i))

In [43]:
vaderAnalysisData_test = review_va_stop_test.map(lambda i: VaderAnalysis(i))

In [44]:
senti_neg_t, senti_neu_t, senti_pos_t, senti_compound_t = [list(t) for t in zip(*vaderAnalysisData_test)]

In [45]:
test['vader_neg'] = senti_neg_t
test['vader_neu'] = senti_neu_t
test['vader_pos'] = senti_pos_t
test['vader_comp'] = senti_compound_t

Add Stanford sentimental analysis feature

In [60]:
from pycorenlp import StanfordCoreNLP

def StanfordAnalysis(inputstring):
    nlp = StanfordCoreNLP('http://localhost:9000')
    totalPositiveCount = 0
    totalNegativeCount = 0
    totalNeutralCount = 0
    totalSentenceCount = 0 
    averagePositiveScore = 0
    averageNegativeScore = 0
    averageNeutralScroe = 0
    
    res = nlp.annotate(inputstring,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 10000,
                   })
    try:
        for s in res["sentences"]:
            sentimentValue = int(s["sentimentValue"])
            sentiment = s["sentiment"]
            
            # negative sentiment
            if sentimentValue < 2:
                totalNegativeCount = totalNegativeCount + 1
            elif sentimentValue == 2:
                totalNeutralCount = totalNeutralCount + 1
            else:
                totalPositiveCount = totalPositiveCount + 1
                    
            totalSentenceCount = totalSentenceCount + 1
                
            if totalSentenceCount > 0:
                averagePositiveScore = totalPositiveCount/totalSentenceCount
                averageNegativeScore = totalNegativeCount/totalSentenceCount
                averageNeutralScroe = totalNeutralCount/totalSentenceCount
    except:
        print ("Error encountered!")
        
    return (averageNegativeScore, averageNeutralScroe, averagePositiveScore)


In [61]:
stanfordSentimental_Analysis = review_va_stop.map(lambda i: StanfordAnalysis(i))

2
0.0 1.0 0.0


1
1.0 0.0 0.0


2
0.0 1.0 0.0


1
1.0 0.0 0.0


1
1.0 0.0 0.0


...

2
0.0 1.0 0.0


0
1.0 0.0 0.0


1
1.0 0.0 0.0


1
1.0 0.0 0.0
1
1.0 0.0 0.0


KeyboardInterrupt: 

In [ ]:
stanford_senti_neg, stanford_senti_neu, stanford_senti_pos = [list(t) for t in zip(*stanfordSentimental_Analysis)]

In [ ]:
train['stanford_neg'] = stanford_senti_neg
train['stanford_neu'] = stanford_senti_neu
train['stanford_pos'] = stanford_senti_pos

In [ ]:
StanfordAnalysisData_test = review_va_stop_test.map(lambda i: StanfordAnalysis(i))

In [ ]:
stanford_senti_neg_t, stanford_senti_neu_t, stanford_senti_pos_t = [list(t) for t in zip(*StanfordAnalysisData_test)]

In [ ]:
test['stanford_neg'] = stanford_senti_neg_t
test['stanford_neu'] = stanford_senti_neu_t
test['stanford_pos'] = stanford_senti_pos_t

In [237]:
test.shape

(53766, 14)

In [239]:
test_recall_df = test.loc[test['recall_status'] == True]
print(test_recall_df.shape)

(10565, 14)


In [240]:
test_nonrecall_df = test.loc[test['recall_status'] == False]
print(test_nonrecall_df.shape)

(43201, 14)


In [241]:
test_recall_df.to_csv("drugsTestDataRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [242]:
test_nonrecall_df.to_csv("drugsTestDataNonRecallOnlyWithFeatures", sep='\t', encoding='utf-8')

In [244]:
test.to_csv("drugsTestingDataWithFeatures", sep='\t', encoding='utf-8')